**Python para Mercado Financeiro, Módulo 7 - Análise Fundamentalista em Python**

**Autor:** Pablo Souza Grigoletti

**E-mail:** psgrigoletti@gmail.com



#1. Instalando e importando as bibliotecas necessárias

In [1]:
# Instalação das bibliotecas
!pip install Plotly
!pip install xlsxwriter
!pip install yfinance
!pip install fundamentus

# Import das bibliotecas
import pandas as pd
import numpy as np
import fundamentus
import yfinance as yf

import requests
import zipfile
import xlsxwriter

import io
import os

import plotly
import plotly.graph_objects as go

import time
from datetime import date

from multiprocessing import Pool

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 1.2 MB/s eta 0:00:001.3 MB/s eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached fundamentus-0.2.0-py3-none-any.whl (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 972.9 kB/s eta 0:00:00 kB/s eta 0:00:01
  Using cached cattrs-22.2.0-py3-none-any.whl (35 kB)
  Using cached

2023-03-09 23:34:00,050 [logging.log_init] INFO: LOGLEVEL=INFO


# 2. Buscando os dados das empresas listadas

Vamos extrair apenas o documento descritivo das companhias de capital aberto extraído da CVM. Com isso, poderemos filtrar as empresas de acordo com a característica que queremos e pegar os códigos de empresa.

##2.1. Baixando os dados

In [2]:
# r = requests.get('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv')
# lines = [i.strip().split(';') for i in r.text.split('\n')]
# df = pd.DataFrame(lines[1:], columns = lines[0])
# df.head()

df_dados_empresas = pd.read_csv('http://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/cad_cia_aberta.csv', sep=";", encoding="ISO-8859-1")
df_dados_empresas.head()

,CNPJ_CIA,DENOM_SOCIAL,DENOM_COMERC,DT_REG,DT_CONST,DT_CANCEL,MOTIVO_CANCEL,SIT,DT_INI_SIT,CD_CVM,...,UF_RESP,PAIS_RESP,CEP_RESP,DDD_TEL_RESP,TEL_RESP,DDD_FAX_RESP,FAX_RESP,EMAIL_RESP,CNPJ_AUDITOR,AUDITOR
0,08.773.135/0001-00,2W ENERGIA S.A.,BANCO BESA SA,2020-10-29,2007-03-23,NaN,NaN,ATIVO,2020-10-29,25224,...,SP,NaN,4707000.0,11,39579400.0,NaN,NaN,ri@2wenergia.com.br,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.
1,11.396.633/0001-87,3A COMPANHIA SECURITIZADORA,TRIPLO A COMPANHIA SECURITIZADORA,2010-03-08,2009-11-03,2015-12-18,Cancelamento Voluntário - IN CVM 480/09,CANCELADA,2015-12-18,21954,...,RJ,NaN,20020000.0,21,22338867.0,21.0,22338867.0,juridico@triploasec.com.br,60.525.706/0001-07,MOORE STEPHENS LIMA LUCCHESI AUDITORES INDEPEN...
2,12.091.809/0001-55,3R PETROLEUM OLÉO E GÁS S.A.,NaN,2020-11-09,2010-06-08,NaN,NaN,ATIVO,2020-11-09,25291,...,RJ,NaN,22250180.0,21,21455555.0,0.0,0.0,ri@3rpetroleum.com.br,57.755.217/0001-29,KPMG AUDITORES INDEPENDENTES LTDA.
3,01.547.749/0001-16,521 PARTICIPAÇOES S.A. - EM LIQUIDAÇÃO EXTRAJU...,521 PARTICIPAÇÕES S/A,1997-07-11,1996-07-30,NaN,NaN,ATIVO,1997-07-11,16330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.596.945/0001-83,BAKER TILLY 4PARTNERS AUDITORES INDEPENDENTES S/S
4,01.851.771/0001-55,524 PARTICIPAÇOES SA,524 PARTICIPACOES SA,1997-05-30,1997-04-02,NaN,NaN,ATIVO,1997-05-30,16284,...,RJ,NaN,20020010.0,21,38043700.0,0.0,0.0,gar@opportunity.com.br,40.262.602/0001-31,"BKR - LOPES, MACHADO AUDITORES"


##2.2. Filtrando o DataFrame

Procurando por empresas específicas, que tenham uma característica em comum na sua denominação comercial.

In [3]:
situacao_interesse = 'ATIVO'
tipo_mercado_interesse = 'BOLSA'

df_dados_empresas_filtradas = df_dados_empresas
df_dados_empresas_filtradas = df_dados_empresas_filtradas[(df_dados_empresas_filtradas['SIT'] == situacao_interesse) & (df_dados_empresas_filtradas['TP_MERC'] == tipo_mercado_interesse)]

# TODO: trocar o código abaixo e encontrar uma forma melhor de filtrar os ativos...
cnpjs_interesse = ['05.878.397/0001-32', '07.816.890/0001-53', '51.218.147/0001-93', '06.977.751/0001-49', '06.977.745/0001-91']
denominacoes_interesse = ['MULTIPLAN', 'MALL', 'IGUATEMI', 'SHOPP']
df_dados_empresas_filtradas = df_dados_empresas_filtradas[df_dados_empresas_filtradas['CNPJ_CIA'].isin(cnpjs_interesse)]
df_dados_empresas_filtradas = df_dados_empresas_filtradas[df_dados_empresas_filtradas.DENOM_SOCIAL.str.contains('|'.join(denominacoes_interesse), na=False)]
df_dados_empresas_filtradas

# Precisamos obter os códigos CVM das empresas que queremos obter.
# Uma vez que sabemos os códigos a partir da tabela anterior, vamos transformar essa coluna em uma lista.
# Já ajustado para ficar com 6 dígitos, preenchidos com zero a esquerda
codigos_cvm = list(df_dados_empresas_filtradas['CD_CVM'])
codigos_cvm = list(map(lambda i: str(i).zfill(6), codigos_cvm))
codigos_cvm

['022357', '019909', '020494', '020982']

# 3. Extraindo as suas demonstrações financeiras

## 3.1. Descrição

Quais as demonstrações que você precisa? O conjunto de dados disponibiliza as seguintes demonstrações financeiras entregues nos últimos cinco anos:
- Balanço Patrimonial Ativo (BPA)
- Balanço Patrimonial Passivo (BPP)
- Demonstração de Fluxo de Caixa - Método Direto (DFC-MD)
- Demonstração de Fluxo de Caixa - Método Indireto (DFC-MI)
- Demonstração das Mutações do Patrimônio Líquido (DMPL)
- Demonstração de Resultado Abrangente (DRA)
- Demonstração de Resultado (DRE)
- Demonstração de Valor Adicionado (DVA)

**OBSERVAÇÃO: Diferença entre balanço patrimonial Consolidado e Individual:**

Nas demonstrações financeiras (DFs) onde encontramos o balanço patrimonial, você vai notar que há duas seções: as DFs individuais e DFs consolidadas. Dentro de cada uma, há diferentes balanços patrimoniais.

A diferença é que o balanço consolidado traz informações referentes à empresa principal e também às subsidiárias (controladas pela empresa principal). 

Já os números do balanço individual se referem somente à "empresa mãe".

Entre os investidores, o mais comum é utilizar as informações consolidadas na análise. Afinal, é a que dá a visão mais realista da posição financeira da companhia.

O balanço patrimonial individual pode ser considerado em casos em que as subsidiárias atuam em segmentos totalmente diferentes da empresa principal e não sofrem grandes intervenções de sua controladora.

##3.2. Baixando os arquivos e ajustando os arquivos (processamento em paralelo)

In [4]:
demonstrativos_interesse = ['BPA', 'DRE', 'BPP']

In [5]:
def retorna_ponteiro_arquivo_zip(url):
    try:
        req = requests.get(url)
        arquivo_zip = zipfile.ZipFile(io.BytesIO(req.content))
        print('Sucesso ao baixar o arquivo de ' + url)
    except:
        print("Erro ao buscar o arquivo de " + url)
        arquivo_zip = None
    return arquivo_zip

def atualiza_dict_df_demonstrativo(tipo, arquivo_zip, demonstrativo, ano, codigos_cvm, dict_df):
    if not arquivo_zip is None:
        nome_arquivo_csv = tipo + '_cia_aberta_' + str(demonstrativo) + '_con_' + str(ano) + '.csv' # CONSIDERANDO SOMENTE OS DF CONSOLIDADOS
        arquivo_csv = arquivo_zip.open(nome_arquivo_csv)
        linhas = arquivo_csv.readlines()
        linhas = [i.strip().decode('ISO-8859-1') for i in linhas]
        linhas = [i.split(';') for i in linhas]
        df = pd.DataFrame(linhas[1:], columns = linhas[0])
        df['VL_AJUSTADO'] = pd.to_numeric(df['VL_CONTA'])
        df = df[df['ORDEM_EXERC']=="ÚLTIMO"]

        for codigo_cvm in codigos_cvm:
            if not codigo_cvm + "|" + demonstrativo in dict_df:
                dict_df[codigo_cvm + "|" + demonstrativo] = df[df['CD_CVM']== str(codigo_cvm)]
            else:
                dict_df[codigo_cvm + "|" + demonstrativo] = pd.concat([dict_df[codigo_cvm + "|" + demonstrativo], 
                                                                       df[df['CD_CVM']== str(codigo_cvm)]])

def processa_arquivos_do_ano(ano):
    url_itr = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_' + str(ano) + '.zip'
    url_dfp = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_' + str(ano) + '.zip'
    arquivo_zip_itr = retorna_ponteiro_arquivo_zip(url_itr)
    arquivo_zip_dfp = retorna_ponteiro_arquivo_zip(url_dfp)

    dict_df = {}

    #for i in range(0, len(codigos_cvm)):
        #codigo_cvm = codigos_cvm[i]
        # writer = pd.ExcelWriter(f'demonstrativos_empresa_{str(codigo_cvm)}_ano_{ano}.xlsx', engine='xlsxwriter')
    for demonstrativo in demonstrativos_interesse:
        
        atualiza_dict_df_demonstrativo("itr", arquivo_zip_itr, demonstrativo, ano, codigos_cvm, dict_df)
        atualiza_dict_df_demonstrativo("dfp", arquivo_zip_dfp, demonstrativo, ano, codigos_cvm, dict_df)
        #df = pd.concat([df_itr, df_dfp])

        #df.to_excel(writer, sheet_name=demonstrativo, encoding= 'ISO-8859-1')
        print(f'Dados (de todas as empresas de interesse) do demonstrativo {demonstrativo} para o ano {ano} foram armazenados.')
        #writer.save()
        #print(f'Arquivo Excel com os demonstrativos da empresa {codigo_cvm} para o ano {ano} já exportado.\n')

    for i in range(0, len(codigos_cvm)):
        codigo_cvm = codigos_cvm[i]
        writer = pd.ExcelWriter(f'demonstrativos_empresa_{str(codigo_cvm)}_ano_{ano}.xlsx', engine='xlsxwriter')        

        for demonstrativo in demonstrativos_interesse:
            dict_df[codigo_cvm + "|" + demonstrativo].to_excel(writer, sheet_name=demonstrativo, encoding= 'ISO-8859-1')
        
        writer.save()
        print(f'Arquivo Excel com os demonstrativos da empresa {codigo_cvm} para o ano {ano} já exportado.')    

start_time = time.time()
anos_interesse = [2019, 2020, 2021, 2022]

with Pool() as pool:
    pool.map(processa_arquivos_do_ano, anos_interesse)
    
print("O tempo de execução foi de %s segundos" % (time.time() - start_time))

Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2019.zip
Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2022.zip
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2022.zip
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2022 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo DRE para o ano 2022 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo BPP para o ano 2022 foram armazenados.


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/tmp/ipykernel_201934/2783785319.py:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Arquivo Excel com os demonstrativos da empresa 022357 para o ano 2022 já exportado.
Arquivo Excel com os demonstrativos da empresa 019909 para o ano 2022 já exportado.
Arquivo Excel com os demonstrativos da empresa 020494 para o ano 2022 já exportado.
Arquivo Excel com os demonstrativos da empresa 020982 para o ano 2022 já exportado.
Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2020.zip
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2020.zip
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2020 foram armazenados.
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2019.zip
Dados (de todas as empresas de interesse) do demonstrativo DRE para o ano 2020 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2019 foram armazenados.
Dados (de todas as empresas de interes

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/tmp/ipykernel_201934/2783785319.py:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Arquivo Excel com os demonstrativos da empresa 022357 para o ano 2020 já exportado.
Arquivo Excel com os demonstrativos da empresa 019909 para o ano 2020 já exportado.
Arquivo Excel com os demonstrativos da empresa 020494 para o ano 2020 já exportado.
Arquivo Excel com os demonstrativos da empresa 020982 para o ano 2020 já exportado.
Dados (de todas as empresas de interesse) do demonstrativo BPP para o ano 2019 foram armazenados.


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/tmp/ipykernel_201934/2783785319.py:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Arquivo Excel com os demonstrativos da empresa 022357 para o ano 2019 já exportado.
Arquivo Excel com os demonstrativos da empresa 019909 para o ano 2019 já exportado.
Arquivo Excel com os demonstrativos da empresa 020494 para o ano 2019 já exportado.
Arquivo Excel com os demonstrativos da empresa 020982 para o ano 2019 já exportado.
Sucesso ao baixar o arquivo de http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_2021.zip
Sucesso ao baixar o arquivo de https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/dfp_cia_aberta_2021.zip
Dados (de todas as empresas de interesse) do demonstrativo BPA para o ano 2021 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo DRE para o ano 2021 foram armazenados.
Dados (de todas as empresas de interesse) do demonstrativo BPP para o ano 2021 foram armazenados.


/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
/tmp/ipykernel_201934/2783785319.py:58: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


Arquivo Excel com os demonstrativos da empresa 022357 para o ano 2021 já exportado.
Arquivo Excel com os demonstrativos da empresa 019909 para o ano 2021 já exportado.
Arquivo Excel com os demonstrativos da empresa 020494 para o ano 2021 já exportado.
Arquivo Excel com os demonstrativos da empresa 020982 para o ano 2021 já exportado.
O tempo de execução foi de 41.11031913757324 segundos


#4. Transformando os dados

##4.1. Verificando os arquivos XLSX gerados

In [6]:
# Capturar os arquivos que estão no diretório criado na etapa do webscrapping
caminho = os.getcwd()
arquivos = os.listdir(caminho)
# arquivos

# Juntamos todos esses xslxs em uma lista
arquivos_xlsx = [f for f in arquivos if f[-4:] == 'xlsx']
arquivos_xlsx

['demonstrativos_empresa_020982_ano_2020.xlsx',
 'demonstrativos_empresa_020494_ano_2022.xlsx',
 'demonstrativos_empresa_020494_ano_2021.xlsx',
 'demonstrativos_empresa_019909_ano_2020.xlsx',
 'demonstrativos_empresa_022357_ano_2019.xlsx',
 'demonstrativos_empresa_019909_ano_2021.xlsx',
 'demonstrativos_empresa_020494_ano_2020.xlsx',
 'demonstrativos_empresa_019909_ano_2019.xlsx',
 'demonstrativos_empresa_022357_ano_2022.xlsx',
 'demonstrativos_empresa_020982_ano_2021.xlsx',
 'demonstrativos_empresa_020494_ano_2019.xlsx',
 'demonstrativos_empresa_019909_ano_2022.xlsx',
 'demonstrativos_empresa_022357_ano_2020.xlsx',
 'demonstrativos_empresa_022357_ano_2021.xlsx',
 'demonstrativos_empresa_020982_ano_2022.xlsx',
 'demonstrativos_empresa_020982_ano_2019.xlsx']

##4.2. Tabela DRE

In [7]:
def adicionar_dados_4_trimestre(dre):
    dre_infos_12_meses = dre[(dre['DT_INI_EXERC'].str[4:8] == '-01-') & (dre['DT_FIM_EXERC'].str[4:8] == '-12-')]
    dre_infos_9_meses = dre[(dre['DT_INI_EXERC'].str[4:8] == '-01-') & (dre['DT_FIM_EXERC'].str[4:8] == '-09-')]

    for index, row in dre_infos_12_meses.iterrows():
        codigo_cvm = row['CD_CVM']
        ordem = row['ORDEM_EXERC']
        codigo_conta = row['CD_CONTA']
        descricao_conta = row['DS_CONTA']
        valor_ajustado_12_meses = row['VL_AJUSTADO']
        nova_data_inicio = row['DT_INI_EXERC'][0:4] + '-10-01'

        try:
            valor_ajustado_9_meses = dre_infos_9_meses[(dre_infos_9_meses['CD_CONTA'] == codigo_conta) & 
                                                       (dre_infos_9_meses['ORDEM_EXERC'] == ordem)].iloc[0]['VL_AJUSTADO']
            #print(f"Achou valor para a conta {codigo_conta} {descricao_conta}")
            #print(f"Valor para os 9 meses: {valor_ajustado_9_meses}")
            #print(f"Valor para os 12 meses: {valor_ajustado_12_meses}")
            #print(f"Vai setar a nova data inicial para: {nova_data_inicio}\n\n")
            row_temp = row.copy()
            row_temp['VL_AJUSTADO'] = valor_ajustado_12_meses - valor_ajustado_9_meses
            row_temp['DT_INI_EXERC'] = nova_data_inicio
            dre = dre.append([row_temp])
        except:
            print(f"Não encontrou valor para a conta {codigo_conta} - '{descricao_conta}' para os 9 meses. Encontrou para os 12.")
    
    return dre

# Criar o dataframe DRE
dre_completo = pd.DataFrame()

# Juntar todos os arquivos xlsx em uma grande tabela DRE
for arquivo_xlsx in arquivos_xlsx:
    dre = pd.read_excel(arquivo_xlsx, sheet_name='DRE')
    dre = adicionar_dados_4_trimestre(dre)
    dre_completo = dre_completo.append(dre)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# Puxar uma lista com os nomes das empresas
lista_de_empresas = dre_completo['DENOM_CIA'].unique().tolist()
lista_de_empresas

In [ ]:
#tickers = ['IGTA3', 'BRML3', 'MULT3', 'ALSO3'] # precisa estar na mesma ordem da lista de empresas

In [ ]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = dre_completo['CD_CONTA'] + " - " + dre_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

In [ ]:
# Organizar a tabela em um esquema de tabela dinamica, onde as datas sejam labels de coluna
dre_completo = pd.pivot_table(dre_completo, index=['DENOM_CIA','DS_CONTA'], columns=['DT_INI_EXERC', 'DT_FIM_EXERC'],values=['VL_AJUSTADO'])
dre_completo.head()

##4.3. Tabelas de Balanço Patrimonal Ativo (BPA) e Balanço Patrimonal Passivo (BPP)

Balanço patrimonial é um documento no qual as empresas divulgam números referentes a seus ativos, passivos e patrimônio líquido. 

Companhias de capital aberto, isto é, com ações negociadas na Bolsa de Valores, são obrigadas a divulgar o balanço patrimonial trimestralmente no Formulário de Informações Trimestrais (ITR) e anualmente no formulário de Demonstrações Financeiras Padronizadas (DFP).

Desse modo, qualquer pessoa pode conferir qual a situação da empresa em que investe ou considera investir em um trimestre específico, ou então no consolidado do ano.

Como o nome já entrega, as DFPs são padronizadas, o que significa que todas as empresas estruturam e classificam as informações da mesma maneira — inclusive os dados referentes ao balanço patrimonial.

Isso torna a experiência de quem analisa múltiplas empresas muito mais fácil, pois é possível encontrar rapidamente as informações desejadas e comparar os dados de diferentes companhias.

Os ITRs e DFPs dividem em dois grupos os números do balanço patrimonial:

###4.3.1. Balanço Patrimonial Ativo (BPA)

Um ativo é um bem ou direito da empresa. Um recurso que, em tese, é útil ou necessário para ajudá-la a obter benefícios econômicos futuros.

O dinheiro em caixa, por exemplo, é um ativo necessário para pagar os funcionários que executam as rotinas administrativas, financeiras e operacionais do negócio.

Mas há diversos tipos de ativos, e o balanço patrimonial os divide entre **ativo circulante** — os bens e direitos que têm maior liquidez, ou seja, podem ser convertidos em dinheiro com mais facilidade e rapidez — e **ativo não circulante** — com menor liquidez.

In [ ]:
# Preparando o BPA
bpa_completo = pd.DataFrame()

for f in arquivos_xlsx:
    bpa = pd.read_excel(f, sheet_name='BPA')
    bpa_completo = bpa_completo.append(bpa)

In [ ]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = bpa_completo['CD_CONTA'] + " - " + bpa_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

In [ ]:
bpa_completo = pd.pivot_table(bpa_completo, index=['DENOM_CIA','DS_CONTA', 'CD_CONTA'], columns=['DT_FIM_EXERC'],values=['VL_AJUSTADO'])
bpa_completo    

###4.3.2. Balanço Patrimonial Passivo (BPP)

O passivo, por outro lado, é uma obrigação, como dívidas e contas a pagar. Enquanto o ativo é um recurso disponível para uso, o passivo é um elemento que representará uma saída de recursos e consequente diminuição do ativo.

Assim como no caso anterior, o balanço patrimonial passivo divide as obrigações entre **passivo circulante** — cujo vencimento ocorre em até 12 meses — e **não circulante** — que pode ser pago em um prazo maior de 12 meses.

Os ativos e passivos circulantes e não circulantes são divididos em outras subcategorias, que apresentaremos mais adiante.

É muito importante acrescentar que as empresas listadas na bolsa acrescentam ao balanço patrimonial passivo o patrimônio líquido, que é o que sobra ao diminuir o passivo do ativo da empresa.

In [ ]:
bpp_completo = pd.DataFrame()

for arquivo_xlsx in arquivos_xlsx:
  bpp = pd.read_excel(arquivo_xlsx, sheet_name='BPP')
  bpp_completo = bpp_completo.append(bpp)

In [ ]:
lista_de_contas = pd.DataFrame()
lista_de_contas["CONTA"] = bpp_completo['CD_CONTA'] + " - " + bpp_completo['DS_CONTA']
lista_de_contas["CONTA"].unique().tolist()

In [ ]:
bpp_completo = pd.pivot_table(bpp_completo, index=['DENOM_CIA','DS_CONTA', 'CD_CONTA'], columns=['DT_FIM_EXERC'], values=['VL_AJUSTADO'])
bpp_completo

#5. Indicadores Fundamentalistas


##5.1. Funções para ajustar DFs e plotar gráficos (anual e trimestral)

In [ ]:
def gera_df_anual(df, filtros_indice):
    df_anual = df.copy(deep=True)

    for filtro in filtros_indice:
        df_anual = df_anual[df_anual.index.get_level_values(filtro['indice']).str[4:8] == filtro['mes']]

    df_anual.index = df_anual.index.get_level_values(1).str[:4]
    return df_anual

def retorna_trimestre(df, possuiDataInicioExercicio=True):
    if (possuiDataInicioExercicio and df['MES_INICIAL'] == '01' and df['MES_FINAL'] == '03') or ((not possuiDataInicioExercicio) and df['MES_FINAL'] == '03'):
        return str(df['ANO']) + '-01-01' 
    if (possuiDataInicioExercicio and df['MES_INICIAL'] == '04' and df['MES_FINAL'] == '06') or ((not possuiDataInicioExercicio) and df['MES_FINAL'] == '06'):
        return str(df['ANO']) + '-04-01' 
    if (possuiDataInicioExercicio and df['MES_INICIAL'] == '07' and df['MES_FINAL'] == '09') or ((not possuiDataInicioExercicio) and df['MES_FINAL'] == '09'):
        return str(df['ANO']) + '-07-01' 
    if (possuiDataInicioExercicio and df['MES_INICIAL'] == '10' and df['MES_FINAL'] == '12') or ((not possuiDataInicioExercicio) and df['MES_FINAL'] == '12'):
        return str(df['ANO']) + '-10-01' 
    else:
        return None

def gera_df_trimestral(df, possuiDataInicioExercicio=True):
    df_trimestral = df.copy(deep=True)

    if possuiDataInicioExercicio:
        df_trimestral['MES_INICIAL'] = df_trimestral.index.get_level_values(1).str[5:7]
        df_trimestral['MES_FINAL'] = df_trimestral.index.get_level_values(2).str[5:7]
    else:
        df_trimestral['MES_INICIAL'] = df_trimestral.index.get_level_values(1).str[5:7]
        df_trimestral['MES_FINAL'] = df_trimestral.index.get_level_values(1).str[5:7]

    df_trimestral['ANO'] = df_trimestral.index.get_level_values(1).str[:4]
    df_trimestral['TRIMESTRE'] = df_trimestral.apply(retorna_trimestre, axis=1, args=(possuiDataInicioExercicio,))

    df_trimestral.index = df_trimestral['TRIMESTRE']
    # df_trimestral[df_trimestral['TRIMESTRE'].str[-6:]=='-10-01']

    df_trimestral = df_trimestral.drop('TRIMESTRE', axis=1)
    df_trimestral = df_trimestral.drop('MES_INICIAL', axis=1)
    df_trimestral = df_trimestral.drop('MES_FINAL', axis=1)
    df_trimestral = df_trimestral.drop('ANO', axis=1)

    df_trimestral.sort_index(ascending=False, inplace=True)
    df_trimestral = df_trimestral[df_trimestral.index.notnull()]
    return df_trimestral

def gera_grafico_anual(df_anual, titulo, descricao_eixo_y, unidade_eixo_y, unidade_y_apos=True):
    fig = go.Figure()

    anos = list(df_anual.index.unique())

    for c in df_anual.columns:

        if unidade_y_apos:
            template_y = "%{value:.2f} " + unidade_eixo_y
            hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": %{y:.2f}" + unidade_eixo_y
        else:
            template_y = unidade_eixo_y + " %{value:.2f}"
            hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": " + unidade_eixo_y + " %{y:.2f} "

        fig.add_trace(go.Bar(x=df_anual.index,
                             y=df_anual[c],
                             name=c, text=df_anual[c], 
                             texttemplate = template_y,
                             hovertemplate = hover_template_y,
                             textposition="inside",
                             textangle=0,
                             textfont={'family': "Arial", 'size': 15, 'color': "Black"},
                             ))
    fig.update_layout(
        yaxis_title=descricao_eixo_y + " (" + unidade_eixo_y + ")",
        xaxis_title="Anos",
        legend_title="Empresas selecionadas:",    
        title={'text': '<b>' + titulo + ' Anual para os anos de ' + ", ".join(anos),
              'y':0.9, 'x':0.5,
              'xanchor': 'center', 'yanchor': 'top'},
        legend=dict(x=0, y=-0.5),
        barmode='group',
        bargap=0.2, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    # Pega a data em que o gráfico foi gerado
    today = date.today().strftime('%d/%m/%Y')

    fig.add_annotation(x=0.9, y=0, text=f"Fonte dos dados: https://dados.cvm.gov.br/ <br>Data da geração: {today}", showarrow=False,
                      xref='paper', yref='paper', 
                      xshift=150, yshift=-130
        , font=dict(size=12, color="grey")
        , align="left"
        )

    fig.show()

def gera_grafico_trimestral(df_trimestral, titulo, descricao_eixo_y, unidade_eixo_y, unidade_y_apos=True):
    fig = go.Figure()

    anos = list(df_trimestral.index.str[0:4].unique())
    anos.sort()

    if unidade_y_apos:
        template_y = "%{value:.2f} " + unidade_eixo_y
        hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": %{y:.2f}" + unidade_eixo_y
    else:
        template_y = unidade_eixo_y + " %{value:.2f}"
        hover_template_y = "<b>" + titulo + "</b><br>Data: %{label}<br>" + descricao_eixo_y + ": " + unidade_eixo_y + " %{y:.2f} "

    for c in df_trimestral.columns:
        fig.add_trace(go.Bar(x=df_trimestral.index,
                            y=df_trimestral[c],
                            name=c, 
                            hovertemplate=hover_template_y,
                            text=df_trimestral[c], 
                            texttemplate=template_y,
                            textposition="inside",
                            textangle=0,
                            textfont={'family': "Arial", 'size': 15, 'color': "Black"},
                            ))
    fig.update_xaxes(tickangle=-80, )

    fig.update_layout(
        #autosize=False,
        #width=1200,
        #height=800,
        yaxis_title=descricao_eixo_y + " (" + unidade_eixo_y + ")",
        xaxis_title="Trimestres/Anos",
        legend_title="Empresas selecionadas:",
        title={
            'text': '<b>' + titulo + ' Trimestral para os anos de ' + ", ".join(anos),
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        legend=dict(x=0, y=-0.8),
        barmode='group',
        bargap=0.2, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1) # gap between bars of the same location coordinate.

    # Pega a data em que o gráfico foi gerado

    # fig.update_xaxes(
    #     showgrid=True,
    #     ticks="outside",
    #     tickson="boundaries",
    #     ticklen=20
    # )

    today = date.today().strftime('%d/%m/%Y')
    fig.add_annotation(x=0.9, y=0, text=f"Fonte dos dados: https://dados.cvm.gov.br/ <br>Data da geração: {today}", showarrow=False,
                      xref='paper', yref='paper', xshift=150, yshift=-130, font=dict(size=12, color="grey"), align="left")

    #fig.update_layout(hovermode="x unified")
    # fig.update_layout(
    #     hoverlabel=dict(
    #         bgcolor="white",
    #         font_size=16,
    #         font_family="Rockwell"
    #     )
    # )
    fig.show()

## 5.2. Margens

### 5.2.1. Margem Bruta (usa a DRE)

#### 5.2.1.1. Descrição

***Referência: https://statusinvest.com.br/termos/m/margem-bruta***

A Margem Bruta é uma métrica utilizada para a identificação da rentabilidade bruta de uma empresa, sendo a razão entre o Lucro Bruto e a Receita Líquida. Assim, através da margem bruta é possível tirar algumas conclusões sobre a eficiência da empresa. Isso ocorre, pois o investidor consegue calcular o quanto a empresa tem de lucro pelos seus investimentos, após a dedução de:

- Impostos sobre Vendas;
- Devoluções;
- Descontos;
- Custos de material;
- Custo de Produto Vendido (CPV).

Para calcular Margem Bruta, basta utilizar a seguinte fórmula:

$MargemBruta = \frac{LucroBruto}{ReceitaLiquida} \times 100$

Os números referentes ao Lucro Bruto e Receita Líquida podem ser recolhidos nos relatórios financeiros oferecidos aos investidores pelas empresas de capital aberto na bolsa. A margem bruta é bastante importante para gestores e investidores, já que produz informações relevantes sobre a rentabilidade da empresa.

#### 5.1.1.2. Ajustando o DataFrame

In [ ]:
margem_bruta_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    resultado_bruto = dre_completo.loc[empresa,:].loc['Resultado Bruto']
    receita = dre_completo.loc[empresa,:].loc['Receita de Venda de Bens e/ou Serviços']
    margem_bruta = pd.Series(resultado_bruto / receita) * 100.0
    margem_bruta_empresas = pd.concat([margem_bruta_empresas, margem_bruta], axis=1)

margem_bruta_empresas.columns = lista_de_empresas

#### 5.1.1.3. Gráfico anual

In [ ]:
gera_grafico_anual(gera_df_anual(margem_bruta_empresas, [{'indice': 1, 'mes': '-01-'}, {'indice': 2, 'mes': '-12-'}]), "Margem Bruta", "Percentual", "%")

#### 5.1.1.4. Gráfico trimestral

In [ ]:
gera_grafico_trimestral(gera_df_trimestral(margem_bruta_empresas), "Margem Bruta", "Percentual", "%")

### 5.2.2. Margem Líquida (usa a DRE)

#### 5.2.2.1. Descrição

***Referência: https://statusinvest.com.br/termos/m/margem-liquida***

A Margem Líquida é razão entre o Lucro Líquido e a Receita Líquida de uma companhia após a dedução de impostos e tributos. Podendo representar um resultado trimestral ou anual, a Margem Líquida representa o resultado líquido das vendas de um negócio. Portanto, está diretamente ligado com o nível de rentabilidade que uma companhia consegue com suas operações. Para os investidores, a Margem Líquida demonstra se uma empresa possui bons retornos a partir custos de produção do seu produto/serviço.

A Margem do Lucro Liquido possibilita aos investidores uma boa avaliação sobre a realidade financeira de uma companhia. Em caso de margem negativa, é possível concluir que a empresa em questão possui prejuízo, já que tanto a receita quanto o lucro depois dos impostos, despesas e custos são levados em conta.

No entanto, é importante lembrar que nem sempre prejuízos significam algo negativo, já que existem algumas variáveis que podem causar esse resultado.

No caso de uma margem alta, a indicação é de que a companhia possui uma boa vantagem competitiva, indicando que seus projetos devem estar dando resultados positivos e é vantajoso o investimento nela. Esse resultado pode ser causado por uma boa exposição da marca ou pelo lançamento de um novo produto ou serviço no mercado, por exemplo.


#### 5.2.2.2. Ajustando o DataFrame

In [ ]:
margem_liquida_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    lucro_liquido = dre_completo.loc[empresa,:].loc['Lucro/Prejuízo Consolidado do Período']
    receita_liquida = dre_completo.loc[empresa,:].loc['Receita de Venda de Bens e/ou Serviços']
    margem_liquida = pd.Series(lucro_liquido / receita_liquida) * 100.0
    margem_liquida_empresas = pd.concat([margem_liquida_empresas, margem_liquida], axis=1)

margem_liquida_empresas.columns = lista_de_empresas

#### 5.2.2.3. Gráfico anual

In [ ]:
gera_grafico_anual(gera_df_anual(margem_liquida_empresas, 
                   [{'indice': 1, 'mes': '-01-'}, {'indice': 2, 'mes': '-12-'}]), 
                   "Margem Líquida", "Percentual", "%")

#### 5.2.2.4. Gráfico trimestral

In [ ]:
gera_grafico_trimestral(gera_df_trimestral(margem_liquida_empresas), 
                        "Margem Líquida", "Percentual", "%")

##5.3. Dívida Bruta/PL (Patrimônio Líquido) (usa o Balanço Patrimonial Passivo - BPP)

###5.3.1. Descrição

***Referência: https://statusinvest.com.br/termos/d/divida-bruta-patrimonio-liquido*** 

Em suma, a **Dívida Bruta/PL** faz um comparativo sobre as questões que envolvem a área de endividamento e alavancagem da empresa, com a sua riqueza total, isto é a diferença entre os passivos e ativos que o negócio possui.

Dessa forma, este se torna um indicador importante no momento de se avaliar a saúde financeira do negócio, pois é possível compreender a capacidade de endividamento que um negócio pode ter, sem comprometer seu valor real.

Afinal, o indicador de **Dívida Bruta/PL** avalia o total de dívidas que o negócio possui sobre o valor real que a empresa tem. Portanto, para o investidor, saber interpretar esse dado é útil no momento de avaliar um investimento, pois tal informação aponta para real saúde financeira da empresa e sua capacidade alavancagem, ou seja, investidores de longo prazo conseguem ter um cenário aproximado sobre o potencial que a empresa tem e o risco que ela pode representar.

### 5.3.2. Ajustando o DataFrame

In [ ]:
pl_ajustado_empresas = pd.DataFrame()

for i in range(0,len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    pl_consolidado = bpp_completo.loc[empresa,:].loc['Patrimônio Líquido Consolidado'].loc["2.03"]
    participacao_nao_controladores = bpp_completo.loc[empresa,:].loc['Participação dos Acionistas Não Controladores'].loc["2.03.09"]
    # Fazer um ajuste no patrimônio líquido para remover a conta 'Participação dos Acionistas Não Controladores'
    pl_ajustado = pd.Series(pl_consolidado - participacao_nao_controladores)     
    pl_ajustado_empresas = pd.concat([pl_ajustado_empresas, pl_ajustado], axis=1)

pl_ajustado_empresas.columns = lista_de_empresas

In [ ]:
# 2.01	Passivo Circulante
# 2.01.04	Empréstimos e Financiamentos
# 2.01.04.01	Empréstimos e Financiamentos

# 2.02	Passivo Não Circulante (prazo de vencimento superior a um ano)	
# 2.02.01	Empréstimos e Financiamentos
# 2.02.01.01	Empréstimos e Financiamentos

divida_bruta_sobre_pl_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    divida_curto_prazo = bpp_completo.loc[empresa,:].loc['Empréstimos e Financiamentos'].loc["2.01.04"]
    divida_longo_prazo = bpp_completo.loc[empresa,:].loc['Empréstimos e Financiamentos'].loc["2.02.01"]
    divida_bruta = pd.Series(divida_curto_prazo + divida_longo_prazo)
    divida_bruta_sobre_pl_empresas = pd.concat([divida_bruta_sobre_pl_empresas, divida_bruta], axis=1)
    
divida_bruta_sobre_pl_empresas.columns = lista_de_empresas
divida_bruta_sobre_pl_empresas = (divida_bruta_sobre_pl_empresas / pl_ajustado_empresas) * 100.0

###5.3.3. Cria o gráfico anual

In [ ]:
gera_grafico_anual(gera_df_anual(divida_bruta_sobre_pl_empresas, [{'indice': 1, 'mes': '-12-'},]), "Dívida Bruta/PL", "Percentual", "%")

###5.3.4. Cria o gráfico trimestral

In [ ]:
gera_grafico_trimestral(gera_df_trimestral(divida_bruta_sobre_pl_empresas, False), "Dívida Bruta/PL", "Percentual", "%")

##5.4. Disponibilidades


###5.4.1. Disponibilidades


####5.4.1.1. Descrição

TODO

#### 5.4.1.2. Ajustando o DataFrame

In [ ]:
caixa_ajustado_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    caixa_ajustado = pd.Series((bpa_completo.loc[empresa,:].loc['Caixa e Equivalentes de Caixa'].loc['1.01.01'] +
                                bpa_completo.loc[empresa,:].loc['Aplicações Financeiras'].loc['1.01.02']))     
    caixa_ajustado_empresas = pd.concat([caixa_ajustado_empresas, caixa_ajustado], axis=1)

caixa_ajustado_empresas.columns = lista_de_empresas

####5.4.1.3. Gráfico anual

In [ ]:
gera_grafico_anual(gera_df_anual(caixa_ajustado_empresas, [{'indice': 1, 'mes': '-12-'}]), 'Disponibilidades', "Valor", "R$", False)

####5.4.1.4. Gráfico trimestral

In [ ]:
# TODO

###5.3.2. Liquidez Corrente

####5.3.2.1. Descrição

***Referência: https://statusinvest.com.br/termos/l/liquidez-corrente***

A liquidez corrente é um indicador usado para mensurar, a curto prazo de tempo, a capacidade de pagamento de uma companhia. Desse modo, a liquidez corrente busca obter a razão entre o ativo circulante da companhia pelo passivo circulante.

Dessa forma, através da liquidez corrente é possível analisar a capacidade de pagamento de uma empresa, que pode ajudar a identificar a saúde do negócio.

A liquidez corrente possui o propósito de mensurar a capacidade de pagamento de uma companhia. Desse modo, existem algumas interpretações da liquidez corrente que possibilitam a compreensão dos resultados, dentre essas interpretações, estão:

- **Maior que 1** – Caso a liquidez corrente apresente um resultado maior que 1, significa que a companhia possui uma capacidade de pagamento relativamente boa, ou seja, expõe que a empresa possui um líquido positivo;

- **Menor que 1** – Entretanto, se o indicativo da liquidez corrente apresentar um resultado inferior a 1,  demonstra que a empresa como dependente de geração de caixa, dado que, o ativo circulante está inferior ao passivo circulante;

- **Igual a 1** – Além disso, caso o indicativo da liquidez corrente apresente um resultado igual a 1, revela que o capital da empresa e seus pagamentos estão proporcionais.


In [ ]:
liquidez_corrente_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    ativo_circulante = bpa_completo.loc[empresa,:].loc['Ativo Circulante'].loc['1.01']
    passivo_circulante = bpp_completo.loc[empresa,:].loc['Passivo Circulante'].loc['2.01']
    liquidez_corrente = pd.Series(ativo_circulante/passivo_circulante)     
    liquidez_corrente_empresas = pd.concat([liquidez_corrente_empresas, liquidez_corrente], axis=1)

liquidez_corrente_empresas.columns = lista_de_empresas
# liquidez_corrente_empresas

##5.5. EBIT

### 5.5.1. Descrição

***Referência: https://statusinvest.com.br/termos/e/ebit***

O **EBIT**, ou *Earnings Before Interest and Taxes*, é um indicador que demonstra o lucro da empresa antes do reconhecimento de suas despesas com juros e impostos.

Em português, o **EBIT** é chamado de LAJIR, ou Lucro Antes de Juros e Impostos. Através dele, é possível entender melhor os resultados operacionais informados no Demonstrativo de Resultados de exercício (DRE) de uma companhia.

Para fazer o cálculo do EBIT é necessário lembrar que esse indicador representa o lucro da empresa antes do pagamento de juros e impostos. Portanto, basta somar os encargos líquidos, impostos sobre lucros e encargos financeiros de financiamento, a partir da seguinte fórmula:

$EBIT=LucroLiquido + ResultadoFinanceiro + Impostos$

Para encontrar essas informações, basta consultar o Demonstrativo de Resultados do Exercício (DRE) da empresa em questão. Esse documento contábil é divulgado trimestralmente pelas companhias de capital aberto na bolsa de valores.

In [ ]:
ebit_ajustado_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    ebit_ajustado = pd.Series(dre_completo.loc[empresa,:].loc['Receita de Venda de Bens e/ou Serviços'] +
                              dre_completo.loc[empresa,:].loc['Custo dos Bens e/ou Serviços Vendidos'] + 
                              dre_completo.loc[empresa,:].loc['Despesas com Vendas'] +
                              dre_completo.loc[empresa,:].loc['Despesas Gerais e Administrativas'])
    ebit_ajustado_empresas = pd.concat([ebit_ajustado_empresas, ebit_ajustado], axis=1)

ebit_ajustado_empresas.columns = lista_de_empresas

In [ ]:
gera_grafico_anual(gera_df_anual(ebit_ajustado_empresas, [{'indice': 1, 'mes': '-01-'},
                                                          {'indice': 2, 'mes': '-12-'}]), "EBIT", "Valor", "R$", False)

In [ ]:
gera_grafico_trimestral(gera_df_trimestral(ebit_ajustado_empresas, True), "EBIT", "Valor", "R$", False)

## Margem Operacional



### Descrição


### Ajustando os DataFrames

In [ ]:
margem_operacional_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    resultado_bruto = dre_completo.loc[empresa,:].loc['Resultado Bruto']
    receita_vendas = dre_completo.loc[empresa,:].loc['Receita de Venda de Bens e/ou Serviços']
    custo_vendas = dre_completo.loc[empresa,:].loc['Custo dos Bens e/ou Serviços Vendidos']
    despesas_vendas = dre_completo.loc[empresa,:].loc['Despesas com Vendas']
    despesas_gerais = dre_completo.loc[empresa,:].loc['Despesas Gerais e Administrativas']

    margem_operacional = pd.Series((resultado_bruto)/(receita_vendas + custo_vendas + despesas_vendas + despesas_gerais))
    margem_operacional_empresas = pd.concat([margem_operacional_empresas, margem_operacional], axis=1)

margem_operacional_empresas.columns = lista_de_empresas

In [ ]:
gera_grafico_anual(gera_df_anual(margem_operacional_empresas, [{'indice': 1, 'mes': '-01-'}, {'indice': 2, 'mes': '-12-'}]), "Margem Operacional", "Percentual", "%", True)

##5.4. Ebitda/Div Bruta

In [ ]:
ebitda_div_bruta_empresas = pd.DataFrame()

for i in range(0, len(lista_de_empresas)):
    empresa = lista_de_empresas[i]
    receita_vendas = dre_completo.loc[empresa,:].loc['Receita de Venda de Bens e/ou Serviços']
    custo = dre_completo.loc[empresa,:].loc['Custo dos Bens e/ou Serviços Vendidos']
    despesas_vendas = dre_completo.loc[empresa,:].loc['Despesas com Vendas']
    despesas_gerais = dre_completo.loc[empresa,:].loc['Despesas Gerais e Administrativas']
    numerador = receita_vendas + custo + despesas_vendas + despesas_gerais
    numerador = numerador[((numerador.index.get_level_values(1).str[4:8] == '-01-') & (numerador.index.get_level_values(2).str[4:8] == '-03-')) |
          ((numerador.index.get_level_values(1).str[4:8] == '-04-') & (numerador.index.get_level_values(2).str[4:8] == '-06-')) |
          ((numerador.index.get_level_values(1).str[4:8] == '-07-') & (numerador.index.get_level_values(2).str[4:8] == '-09-')) |
          ((numerador.index.get_level_values(1).str[4:8] == '-10-') & (numerador.index.get_level_values(2).str[4:8] == '-12-'))
         ]
    #numerador["DELETAR"] = numerador["DT_FIM_EXERC"] - numerador["DT_INI_EXERC"] 

    divida_curto_prazo = bpp_completo.loc[empresa,:].loc['Empréstimos e Financiamentos'].loc["2.01.04"]
    divida_longo_prazo = bpp_completo.loc[empresa,:].loc['Empréstimos e Financiamentos'].loc["2.02.01"]
    denominador = divida_curto_prazo + divida_longo_prazo

    ebit_div_ajustado = pd.Series(numerador/denominador)
    ebitda_div_bruta_empresas = pd.concat([ebitda_div_bruta_empresas, ebit_div_ajustado], axis=1)

ebitda_div_bruta_empresas.columns = lista_de_empresas
#ebitda_div_bruta_empresas

ebitda_div_bruta_empresas
#gera_df_anual(ebitda_div_bruta_empresas, [{'indice': 1, 'mes': '-01-'}, {'indice': 2, 'mes': '-12-'}])

In [ ]:
gera_grafico_anual(gera_df_anual(ebitda_div_bruta_empresas, [{'indice': 1, 'mes': '-01-'}, {'indice': 2, 'mes': '-12-'}]), "EBIT/Divida Bruta", "Percentual", "%", True)

##5.5. Return on Equity - ROE



In [ ]:
#ROE E ROIC 


In [ ]:
#ROE
#Junta os lucros líquidos dos últimos 4 trimestre e divide pelo PL ajustado

##5.6. EBIT/Ativo


In [ ]:
#Roic
receita_liq_Q420 = pd.DataFrame()

for i in range(0,n_empresas):
  receita_liq_Q420_ = pd.Series(dre3.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-1] - (
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-5]+
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-2]+
    dre2.loc[lista_de_empresas[i],:].loc['Resultado Antes do Resultado Financeiro e dos Tributos'].iloc[-1]
))
  receita_liq_Q420 = pd.concat([receita_liq_Q420,receita_liq_Q420_], axis=1)
receita_liq_Q420.columns = lista_de_empresas
receita_liq_Q420

In [ ]:
receita_liq_Q320 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q320_ = pd.Series(dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])
  receita_liq_Q320 = pd.concat([receita_liq_Q320,receita_liq_Q320_], axis=1)
receita_liq_Q320.columns = lista_de_empresas
receita_liq_Q320




In [ ]:
receita_liq_Q221 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q221_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])
  receita_liq_Q221 = pd.concat([receita_liq_Q221,receita_liq_Q221_], axis=1)
receita_liq_Q221.columns = lista_de_empresas
receita_liq_Q221


In [ ]:
receita_liq_Q121 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q121_ = pd.Series(dre.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-3,]+
                             dre.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-3,]+ 
                             dre.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-3,]+
                             dre.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-3,])
  receita_liq_Q121 = pd.concat([receita_liq_Q121,receita_liq_Q121_], axis=1)
receita_liq_Q121.columns = lista_de_empresas
receita_liq_Q121


In [ ]:
receita_liq_Q420 = pd.DataFrame()


for i in range(0,n_empresas):
  receita_liq_Q420_ = pd.Series((dre3.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre3.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre3.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre3.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])-
                             ((dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-1,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-1,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-1,])+
                             (dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-2,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-2,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-2,]+
                             (dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-5,]))+
                             (dre2.loc[lista_de_empresas[i],:].loc['Receita de Venda de Bens e/ou Serviços'].iloc[-5,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Custo dos Bens e/ou Serviços Vendidos'].iloc[-5,]+ 
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas com Vendas'].iloc[-5,]+
                             dre2.loc[lista_de_empresas[i],:].loc['Despesas Gerais e Administrativas'].iloc[-5,]))
                             )
  receita_liq_Q420 = pd.concat([receita_liq_Q420,receita_liq_Q420_], axis=1)
receita_liq_Q420.columns = lista_de_empresas
receita_liq_Q420

In [ ]:
ebit_ano = receita_liq_Q121+receita_liq_Q221+receita_liq_Q420+receita_liq_Q320
ebit_ano

In [ ]:
#EBIT/ATIVO

ebit_ativo = pd.DataFrame()
for i in range(0,n_empresas):
  ebit_ativo_ = pd.Series(bpa.loc[lista_de_empresas[i],:].loc['Ativo Total'].iloc[-1])
  ebit_ativo = pd.concat([ebit_ativo,ebit_ativo_], axis=1)
ebit_ativo.columns = lista_de_empresas
ebit_ativo = ebit_ano/ebit_ativo
ebit_ativo

fig = go.Figure(go.Bar(
    x=ebit_ativo.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='EBIT/ATIVO')
fig.show()

In [ ]:
bpa.loc[lista_de_empresas[i],:].loc['Caixa e Equivalentes de Caixa'].iloc[-1,] + bpa.loc[lista_de_empresas[i],:].loc['Aplicações Financeiras'].iloc[-1,]

##5.7. ROIC

In [ ]:
roic = ebit_ano/(pl_ajustado+divida_bruta_pl_df+caixa_ajustado)
roic

fig = go.Figure(go.Bar(
    x=roic.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='ROIC')
fig.show()

#6. Equity e Preço

Indicadores fundamentalistas ligados ao preço e valor do ativo.


##6.1. Valor de Mercado (Marketcap)

###6.1.1. Descrição

***Referência: https://statusinvest.com.br/termos/v/valor-de-mercado***

O valor de mercado, refere-se ao preço que o mercado está pagando por uma determinada empresa, baseando-se na lei da oferta e da procura e concorrência de mercado.

Através do valor de mercado, será possível verificar o posicionamento de uma organização, perante diversas variáveis que podem influenciar de forma positiva ou negativa o valor das ações.

Ou seja, observa-se que o valor de mercado, verifica o preço que a empresa está sendo negociada no mercado financeiro, durante um período determinado. Vale ressaltar que esse valor varia ao longo do tempo, baseado normalmente no preço em que as ações negociam.

O cálculo do valor de mercado é realizado através da multiplicação do valor de uma ação, pelo número de ações existentes.

Desse modo, o valor de mercado poderá ser calculado mediante a utilização da seguinte fórmula:

$ValorDeMercado = ValorDaAcao \times NumeroDeAcoesExistentes$

O valor de mercado está situado como um dos indicadores mais utilizados para verificar quanto o mercado está avaliando uma empresa. Entretanto, esse número possui algumas limitações, dado que, o valor de negociação reflete apenas uma estimativa do valor da companhia pelos seus acionistas, não de fato seu valor intrínseco.

Dessa forma, é interessante que cada investidor realize sua própria análise a fim de identificar um valor justo para se negociar determinado ativo, sem necessariamente utilizar a expectativa do mercado ao avaliar determinada empresa.

Dessa forma, é válido utilizar o valor de mercado, em conjunto, a outros indicadores para obter uma análise mais realista.



###6.1.2. Ajustando os DataFrames

In [ ]:
# Coletar dados de preços
tickers = ['IGTI3', 'BRML3', 'MULT3', 'ALSO3'] # precisa estar na mesma ordem da lista de empresas

marketcap_empresas = pd.DataFrame()

for i in tickers:
    t = yf.Ticker(i + '.SA')
    historico = t.history(period="1mo", interval="1d")
    ultimo_fechamento = 0
    if len(historico) > 0:
        ultimo_fechamento = historico['Close'][-1]

    numero_acoes = float(fundamentus.get_papel(i).loc[i].loc["Nro_Acoes"])
    valor_marketcap = numero_acoes * ultimo_fechamento
    marketcap = pd.DataFrame(data={i: [valor_marketcap]})
    
    marketcap_empresas = pd.concat([marketcap_empresas, marketcap], axis=1)

marketcap_empresas

###6.1.3. Gráfico anual

In [ ]:
# fig = go.Figure(go.Bar(
#     x=marketcap_empresas.iloc[-1],
#     y=tickers,
#     orientation='h',text=marketcap_empresas.iloc[-1]
# ))
# fig.update_layout(title_text='Marketcap')
# fig.show()

###6.1.4. Gráfico trimestral

In [ ]:
# TODO

##6.2. Valor da Firma (Enterprise Value)

###6.2.1. Descrição

***Referência: https://statusinvest.com.br/termos/v/valor-de-firma***

O valor de firma ou valor do empreendimento, refere-se a quanto vale uma empresa de forma abrangente, visto que, o valor de firma leva consideração, tanto o valor de mercado, como também, os ativos e passivos da organização, definindo assim, quanto realmente vale a companhia.

Além disso, o valor de firma consegue definir quanto uma organização realmente vale, ou seja, quanto custa para adquirir uma empresa e todos os seus ativos, levando em consideração, que o comprador terá que assumir a dívida líquida da empresa.

Sendo assim, o valor de firma é um valor bastante utilizado para comparar empresas.

O valor de firma pode ser usado para interpretar se uma empresa está com um alto valor, quando comparado aos seus concorrentes. O valor de firma proporciona uma interpretação mais detalhada e completa do valor de uma organização.

Dessa forma,  esse valor tornou-se um dos mais utilizados para negociações de aquisições, visto que, o índice pode apresentar a sustentabilidade da empresa nos anos posteriores, lembrando que, o valor de firma leva em consideração as dívidas da empresa e suas reservas em caixa.

O cálculo do valor de firma leva em consideração a capitalização da empresa, somando-se ao seu endividamento, menos o seu caixa e equivalentes, podendo ser revelado mediante a realização da seguinte operação:

$VF=CM+DT-C-AtivosNO$

Logo, as seguintes incógnitas referem-se a:

- **Valor de firma (VF)**: define quanto vale uma organização;
- **Capitalização de mercado (CM)**: refere-se ao valor de mercado da empresa;
- **Dívida total (DT)**: valor referente às dívidas de curto e longo prazo da organização, lembrando que, são considerados apenas os passivos financeiros;
- **Caixa e equivalentes (C)**: totalidade do valor existente no caixa da empresa.
- **Ativos Não Operacionais (AtivosNO)**: ativos que não tem relação direta com o funcionamento da companhia.



###6.2.2. Ajustando os DataFrames

In [ ]:
# ev = marketcap + (divida_bruta_pl_df-caixa_ajustado)

###6.2.3. Gráfico anual

In [ ]:
# ev

# fig = go.Figure(go.Bar(
#     x=ev.iloc[-1],
#     y=tickers,
#     orientation='h'
# ))
# fig.update_layout(title_text='Enterprise Value')
# fig.show()

###6.2.4. Gráfico trimestral

In [ ]:
# TODO

##6.3. Valor Patrimonial da Ação

In [ ]:
#Valor Patrimonial da Ação

vpa = (pl_ajustado)/n_acoes

In [ ]:
vpa

In [ ]:
fig = go.Figure(go.Bar(
    x=vpa.iloc[-1],
    y=tickers,
    orientation='h'
))
fig.update_layout(title_text='Valor Patrimonial da Ação')
fig.show()

In [ ]:
type(pl_ajustado.iloc[-1,-1])

##6.4. Lucro por Ação e P/L

In [ ]:
#LPA

lpa = (Q320+Q420+Q121+Q221)/n_acoes
lpa

In [ ]:
#P/L

pl = shoppings_preços.iloc[-1]/lpa
pl

In [ ]:
fig = go.Figure(go.Bar(
    x=pl.iloc[-1],
    y=tickers,
    orientation='h',
))
fig.update_layout(title_text='Preço/Lucro')
fig.show()

In [ ]:
#P/VPA
pvpa = shoppings_preços.iloc[-1]/vpa
pvpa

In [ ]:


fig = go.Figure(go.Bar(
    x=pvpa.iloc[-1],
    y=tickers,
    orientation='h',text=pvpa.iloc[-1]
))
fig.update_layout(title_text='P/VPA')
fig.show()

In [ ]:

fig = ff.create_table(lpa, height_constant=50)
trace1=go.Bar(x=tickers, y=pl.iloc[-1],marker=dict(color='#0099ff'), name='P/L',xaxis='x2' ,yaxis='y2')
fig.add_traces([trace1])
# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .45]})
fig.layout.yaxis2.update({'domain': [.5, 1]})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Goals'})

fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'title': 'Analitcs'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':800})


fig.show()

# PyGWalker

In [ ]:
!pip install pygwalker
!pip install yfinance
import pygwalker as pyg
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
df = yf.download("PETZ3.SA", period="max")

In [ ]:
df["Retornos"] = df["Adj Close"].pct_change(1)

In [ ]:
# Vol em 10 dias
df["Vol_10_dias"] = df["Retornos"].rolling(10).std()

In [ ]:
# Média móvel 20 dias
df["MM_20_dias"] = df["Adj Close"].rolling(20).mean()

In [ ]:
# Ano e Mẽs
df["Ano"] = df.index.year
df["Mes"] = df.index.month

In [ ]:
df

In [ ]:
import pygwalker as pyg
pyg.walk(df)

In [ ]:
#Api Banco Mundial

!pip install wbgapi
import wbgapi as wb

In [ ]:
lista = ['ARG', 'BRA', "CHL", "COL", "PER", "URY", "MEX"]
desemprego = wb.data.DataFrame("SL.UEM.TOTL.ZS", economy=lista, time=range(2010, 2022))
desemprego

In [ ]:
desemprego = desemprego.melt(var_name="Ano", value_name="Desemprego")
desemprego["País"] = np.resize(lista, len(desemprego))
desemprego["Ano"] = pd.to_numeric(desemprego["Ano"].str.slice(start=2))
desemprego["Desemprego"] = pd.to_numeric(desemprego["Desemprego"])
desemprego

In [ ]:
pyg.walk(desemprego)